In [2]:
import numpy as np
import pandas as pd
import sympy as sp 
from scipy.optimize import minimize 
from scipy.optimize import differential_evolution

In [9]:
f= open('linsep.txt','r')
data = []
for line in f.readlines():
    datapoint = line.strip().split(',')
    datapoint = list(map(float,datapoint))
    data.append(datapoint)

In [10]:
#Check our dataset:
#data

In [11]:
#Create the list for containing lambdas 
lambdas = []
for i in range(len(data)):
    globals()['lambda'+str(i+1)] = sp.symbols('lambda{}'.format(i+1))
    lambdas.append(globals()['lambda'+str(i+1)])
    

In [12]:
#lambdas

In [7]:
#Constructing the duality lagrange function:
sigma_ll_yy_xx = 0 
for i in range(len(data)):
    for j in range(len(lambdas)):
        sigma_ll_yy_xx += lambdas[i] * lambdas[j] * data[i][2] * data[j][2] * np.dot(data[i][0:2],data[j][0:2])

sigma_lambda = 0

for lambda_i in lambdas:
    sigma_lambda += lambda_i

#This equation is our minimization target:
lagrange = (1/2)*sigma_ll_yy_xx-(sigma_lambda)


In [ ]:
#Write the duality lagrange function into the general python function form
def lagrange_min(x):
    zipped = zip(lambdas,x)
    return lagrange.subs(zipped)

In [67]:
#KKT:
#Sum of lambdai x yi must be zero:
def sigma_lambda_y(x):
    sigma_lambda_y = 0
    for i in range(len(lambdas)):
        sigma_lambda_y  += lambdas[i]*data[i][2]-0  
    zipped = zip(lambdas,x)
    return sigma_lambda_y.subs(zipped)

In [78]:
#KKT:
#All lambda value must be no less than zero:
lamb_positive = []
for i in range(len(lambdas)):
    globals()['bound'+str(i+1)]= (0,np.inf)
    lamb_positive.append(globals()['bound'+str(i+1)])

lamb_positive = tuple(lamb_positive)

In [1]:
#Minimize the Lagrangian function:
x0=[2]*100
solution_lambda = minimize(lagrange_min,x0,tol=1e-8,method='COBYLA',bounds=lamb_positive,constraints=[{'type':'eq','fun':sigma_lambda_y}])

In [ ]:
#Second attempt
solution_lambda2 = minimize(lagrange_min,x0,tol=1e-8,method='nelder-mead')

In [ ]:
#Third attempt, using differential_evolution
solution_lambda3 = differential_evolution(lagrange,bounds=lamb_positive)

In [3]:
#solution_lambda.x

In [142]:
#W*

w_star = []

for i in range(len(data)):
    lambdai_yi_xi = [(list(solution_lambda.values())[i]*data[i][2])*x for x in data[i][0:2]]
    w_star = np.sum([w_star,lambdai_yi_xi],axis=0)

w_star = w_star.tolist()